# Biredctional LSTM

In [ ]:
import json
import numpy as np
import re
import io
import nltk
import h5py
import keras as k
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout, RepeatVector, Activation, merge, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import concatenate
from sklearn.metrics import f1_score

CREATING EMBEDING LAYER WITH HELP OF WORD EMBEDDINGS: GLOVE

In [4]:
embeddings_index = {}
f = open( 'glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


IMPORT VECTORIZED FILES FOR CONTEXT, QUESTIONS, ANSWERS POST PREPROCESS OF DATA

In [5]:
context = h5py.File('context.h5','r')
questions = h5py.File('questions.h5','r')
answers = h5py.File('answers.h5','r')
ans_begin = h5py.File('begin.h5','r')
ans_end = h5py.File('end.h5','r')

In [6]:
c_data = context['context'][:]
qn_data = questions['questions'][:]
ans_data = answers['answers'][:]

In [7]:
begin_ans = ans_begin['begin'][:]
end_ans = ans_end['end'][:]

In [8]:
# loding vocabulary
word_index = np.load('words.npy').item()

In [9]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [10]:
vocab_size = len(word_index) + 1
#embedding_vector_length = 50
batch = 64
max_span_begin = np.amax(begin_ans)
max_span_end = np.amax(end_ans)
slce = 10000


In [11]:
print("Vocab Size")
vocab_size

Vocab Size


119616

MODEL CREATION

In [12]:
context_input = Input(shape=(700, ), dtype='int32', name='c_data')
x = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=700, trainable=False)(context_input)
lstm_out = Bidirectional(LSTM(256, return_sequences=True, implementation=2), merge_mode='concat')(x)
drop_1 = Dropout(0.5)(lstm_out)

In [13]:
ques_input = Input(shape=(100, ), dtype='int32', name='qn_data')
x = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=100, trainable=False)(ques_input)
lstm_out = Bidirectional(LSTM(256, return_sequences=True, implementation=2), merge_mode='concat')(x)
drop_2 = Dropout(0.5)(lstm_out)

In [14]:
merge_layer = concatenate([drop_1, drop_2], axis=1)
biLSTM = Bidirectional(LSTM(512, implementation=2), merge_mode='mul')(merge_layer)
drop_3 =  Dropout(0.5)(biLSTM)
softmax_1 = Dense(max_span_begin, activation='softmax')(biLSTM)
softmax_2 = Dense(max_span_end, activation='softmax')(biLSTM)
model = Model(inputs=[context_input, ques_input], outputs=[softmax_1, softmax_2])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c_data (InputLayer)             (None, 700)          0                                            
__________________________________________________________________________________________________
qn_data (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 700, 100)     11961600    c_data[0][0]                     
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 100)     11961600    qn_data[0][0]                    
__________________________________________________________________________________________________
bidirectio

In [15]:
model_history = model.fit([c_data[:slce], qn_data[:slce]],
                        [begin_ans[:slce], end_ans[:slce]], verbose=2,
                         batch_size=batch, epochs=10)

Epoch 1/10
 - 745s - loss: 14.0046 - dense_1_loss: 6.9654 - dense_2_loss: 7.0393 - dense_1_acc: 0.0268 - dense_2_acc: 0.0043
Epoch 2/10
 - 720s - loss: 13.2295 - dense_1_loss: 6.5742 - dense_2_loss: 6.6554 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 3/10
 - 713s - loss: 13.1647 - dense_1_loss: 6.5377 - dense_2_loss: 6.6270 - dense_1_acc: 0.0298 - dense_2_acc: 0.0067
Epoch 4/10
 - 712s - loss: 13.0802 - dense_1_loss: 6.4917 - dense_2_loss: 6.5885 - dense_1_acc: 0.0375 - dense_2_acc: 0.0075
Epoch 5/10
 - 713s - loss: 12.9418 - dense_1_loss: 6.4181 - dense_2_loss: 6.5237 - dense_1_acc: 0.0390 - dense_2_acc: 0.0102
Epoch 6/10
 - 712s - loss: 12.8121 - dense_1_loss: 6.3521 - dense_2_loss: 6.4599 - dense_1_acc: 0.0395 - dense_2_acc: 0.0104
Epoch 7/10
 - 712s - loss: 12.6831 - dense_1_loss: 6.2867 - dense_2_loss: 6.3963 - dense_1_acc: 0.0407 - dense_2_acc: 0.0123
Epoch 8/10
 - 712s - loss: 12.5055 - dense_1_loss: 6.1994 - dense_2_loss: 6.3061 - dense_1_acc: 0.0385 - dense_2_acc: 0.0139


IMPORT VALIDATION DATA POST PREPROCESSING FOR TESTING

In [34]:
t_context = h5py.File('context_test.h5','r')
t_questions = h5py.File('questions_test.h5','r')
t_answers = h5py.File('answers_test.h5','r')
t_ans_begin = h5py.File('begin_test.h5','r')
t_ans_end = h5py.File('end_test.h5','r')

In [40]:
t_c_data = t_context['context'][:]
t_qn_data = t_questions['questions'][:]
t_ans_data = t_answers['answers'][:]
t_begin_ans = t_ans_begin['begin'][:]
t_end_ans = t_ans_end['end'][:]

In [18]:
predictions = model.predict([t_c_data,t_qn_data], batch_size=128)

In [19]:
print(predictions[0].shape, predictions[1].shape)

(20302, 3126) (20302, 3136)


MODEL PERFORMANCE EVALUATION:
    METRICS USED:
        1. F1 SCORE
        2. EXACT MATCH SCORE

In [20]:
ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 

In [21]:
for i in range(predictions[0].shape[0]):
	ansBegin[i] = predictions[0][i, :].argmax()
	ansEnd[i] = predictions[1][i, :].argmax()
print(ansBegin.min(), ansBegin.max(), ansEnd.min(), ansEnd.max())

0 560 7 779


F1 Score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0

In [63]:
f1_b = f1_score(t_begin_ans, ansBegin, average='micro')  

In [64]:
f1_e = f1_score(t_end_ans, ansEnd, average='micro') 

In [66]:
print("F1 Score")
f1_b + f1_e

F1 Score


0.018421830361540735

Exact Match Score: One way of calculating it is through Accuracy classification score.
It computes subset accuracy: the set of answers predicted for a sample data must exactly match the corresponding set of labels in true answers.

In [69]:
em_b = accuracy_score(t_begin_ans, ansBegin)

In [70]:
em_e = accuracy_score(t_end_ans, ansEnd)

In [71]:
print("Exact Match Score")
em_b + em_e

Exact Match Score


0.018421830361540735